In [1]:
import os
import psycopg2
from openai import OpenAI
import json
import time
import re

# Configure OpenAI API
llm = OpenAI(
    api_key="sk-16a90ba86cfcxxxxxx",  # Ensure to secure your API key
    base_url="https://api.deepseek.com"
)

# PostgreSQL Database Connection Parameters
DATABASE = {
    'database': os.getenv('DB_NAME', 'maude'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'xxxxxx'),
    #'host': os.getenv('DB_HOST', '192.168.8.167'),
    'host': os.getenv('DB_HOST', 'server.natappfree.cc'),
    #'port': os.getenv('DB_PORT', '5432')
    'port': os.getenv('DB_PORT', '33755')
}
SCHEMA = 'maude'

def generate_response(prompt):
    """
    Generate a response from OpenAI based on the provided prompt.
    """
    try:
        response = llm.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt},
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None

def read_prompt_file(file_path):
    """
    Read the content of the prompt file.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

def connect_database():
    """
    Establish a connection to the PostgreSQL database.
    """
    try:
        conn = psycopg2.connect(**DATABASE)
        cursor = conn.cursor()
        print("Successfully connected to the database.")
        return conn, cursor
    except Exception as e:
        print(f"Database connection failed: {e}")
        return None, None

def get_table_structure(cursor, table_name):
    """
    Retrieve the structure of a specified table.
    """
    try:
        query = f"""
        SELECT 
            column_name, 
            data_type, 
            character_maximum_length, 
            is_nullable
        FROM 
            information_schema.columns
        WHERE 
            table_schema = '{SCHEMA}' 
            AND table_name = '{table_name}';
        """
        cursor.execute(query)
        columns = cursor.fetchall()
        structure = []
        for col in columns:
            structure.append({
                'column_name': col[0],
                'data_type': col[1],
                'character_max_length': col[2],
                'is_nullable': col[3]
            })
        return structure
    except Exception as e:
        print(f"Error retrieving table structure ({table_name}): {e}")
        return None

def get_sample_data(cursor, table_name, limit=3):
    """
    Retrieve sample data from a specified table.
    """
    try:
        query = f'SELECT * FROM {SCHEMA}."{table_name}" LIMIT {limit};'
        cursor.execute(query)
        rows = cursor.fetchall()
        # Retrieve column names
        col_names = [desc[0] for desc in cursor.description]
        sample_data = [dict(zip(col_names, row)) for row in rows]
        return sample_data
    except Exception as e:
        print(f"Error retrieving sample data ({table_name}): {e}")
        return None

def execute_sql(cursor, sql_query):
    """
    Execute an SQL query and handle the results.
    """
    try:
        cursor.execute(sql_query)
        # If it's a SELECT query, fetch and return the results
        if sql_query.strip().upper().startswith("SELECT"):
            rows = cursor.fetchall()
            col_names = [desc[0] for desc in cursor.description]
            data = [dict(zip(col_names, row)) for row in rows]
            return data, None
        else:
            # For non-SELECT queries, commit the transaction
            cursor.connection.commit()
            return None, None
    except Exception as e:
        return None, str(e)

def extract_table_names(prompt_content):
    """
    Extract real table names from the prompt content.
    Assumes that table names are mentioned in the format `merged table xxx` and replaces them with actual table names.
    For demonstration, uses a predefined mapping.
    """
    # Define a mapping from merged table names to actual table names
    # This should be updated based on actual mappings
    merged_to_real = {
        "Merged_Table_1": "mdrfoi",
        "Merged_Table_2": "patientproblemcode",
        "Merged_Table_3": "some_other_table",  # Replace with actual table names
        # Add all necessary mappings here
    }
    
    # Extract merged table names using regex
    merged_tables = re.findall(r'Merged_Table_\d+', prompt_content)
    
    # Replace merged table names with real table names
    involved_tables = []
    for mt in merged_tables:
        real_table = merged_to_real.get(mt, None)
        if real_table:
            involved_tables.append(real_table)
        else:
            print(f"No real table mapping found for {mt}. Please update the mapping.")
    
    # If no merged tables found, use default involved tables
    if not involved_tables:
        involved_tables = ["mdrfoi", "DEVICE", "deviceproblemcodes", "patient", "patientproblemcode", "foitext", "DISCLAIM"]
    
    return involved_tables

def analyze_data(research_question, data):
    """
    Analyze the data to validate the research question.
    This function can be expanded based on specific analysis requirements.
    For demonstration, it sends the data and research question to OpenAI for analysis.
    """
    if not data:
        print("No data available for analysis.")
        return
    
    # Convert data to JSON string
    data_json = json.dumps(data, ensure_ascii=False)
    
    # Create analysis prompt
    analysis_prompt = (
        f"Based on the following research question and data, analyze the validity and feasibility of the research question.\n\n"
        f"Research Question: {research_question}\n\n"
        f"Data: {data_json}\n\n"
        f"Provide a detailed analysis report:"
    )
    
    # Get analysis report from OpenAI
    analysis_report = generate_response(analysis_prompt)
    if analysis_report:
        print(f"Analysis Report:\n{analysis_report}")
    else:
        print("Failed to generate analysis report.")



In [2]:
# 1. Read table information from prompt.txt
prompt_file = 'prompt.txt'
prompt_content = read_prompt_file(prompt_file)
if not prompt_content:
    print("Failed to read prompt.txt file.")

# 2. Generate a research question
research_prompt = (
    f"Based on the following MAUDE database table information, propose a meaningful research question.\n\n"
    f"{prompt_content}"
)
research_question = generate_response(research_prompt)
if not research_question:
    print("Failed to generate a research question.")
print(f"Proposed Research Question:\n\n{research_question}\n")

Proposed Research Question:

### Research Question:

**How do the rates of adverse events and device problems vary across different types of medical devices, and what factors (such as device age, manufacturer, and type of operator) are associated with higher incidence of these events?**

### Rationale:

This research question is designed to leverage the comprehensive data available in the MAUDE database to explore the following key aspects:

1. **Variation in Adverse Events by Device Type**: The MAUDE database contains detailed information on various types of medical devices, including their generic names, product codes, and brand names. By analyzing the frequency and types of adverse events reported for different device categories, we can identify which types of devices are more prone to causing harm or malfunctioning.

2. **Impact of Device Age**: The database includes information on the age of the device at the time of the event. Understanding how the age of a device correlates with

In [27]:
# 3. Plan execution steps based on the research question
planning_prompt = (
    f"Based on MAUDE Database background information and the following research question, outline specific execution steps, including which tables and fields need to be queried.\n\n\n\n"
    f"Research Question: {research_question}"
)
execution_steps = generate_response(planning_prompt)
if not execution_steps:
    print("Failed to plan execution steps.")
print(f"Planned Execution Steps:\n\n{execution_steps}\n")

2024-12-08 09:37:22,180 [INFO] HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


Planned Execution Steps:

### Execution Steps for Research Question Analysis

To address the research question regarding the rates of adverse events and device problems across different types of medical devices and the factors associated with higher incidence, the following steps outline the specific execution plan, including which tables and fields need to be queried from the MAUDE (Manufacturer and User Facility Device Experience) database.

#### Step 1: Data Collection and Initial Query

1. **Identify Relevant Tables**:
   - **`device` Table**: Contains information about the device, including device type, manufacturer, and device age.
   - **`event` Table**: Contains details about the adverse events and device problems reported.
   - **`manufacturer` Table**: Contains information about the device manufacturers.
   - **`operator` Table**: Contains information about the type of operator involved in the event.

2. **Extract Relevant Fields**:
   - From the **`device` Table**:
     - `d

In [28]:
# 4. Connect to the database
conn, cursor = connect_database()
if not conn or not cursor:
    print("database connection failed")

# 5. Identify involved tables by extracting from execution steps
involved_tables = extract_table_names(execution_steps)
print(f"Identified Involved Tables: {involved_tables}\n")

Successfully connected to the database.
Identified Involved Tables: ['mdrfoi', 'DEVICE', 'deviceproblemcodes', 'patient', 'patientproblemcode', 'foitext', 'DISCLAIM']



In [29]:
# 6. Check table structures and sample data
table_info = {}
for table in involved_tables:
    # Check if table exists
    structure = get_table_structure(cursor, table)
    if structure is None or not structure:
        print(f"Table '{table}' does not exist or has no columns. Skipping.")
    samples = get_sample_data(cursor, table)
    if samples is None:
        print(f"Unable to retrieve sample data for table: {table}")
    table_info[table] = {
        'structure': structure,
        'samples': samples
    }
    print(f"Table: {table}")
    print(f"Structure: {json.dumps(structure, ensure_ascii=False, indent=2)}")
    print(f"Sample Data: {json.dumps(samples, ensure_ascii=False, indent=2)}\n")

if not table_info:
    print("No valid tables found for analysis. Exiting.")
    cursor.close()
    conn.close()

# Convert table information to JSON for OpenAI processing
table_info_json = json.dumps(table_info, ensure_ascii=False)

Table: mdrfoi
Structure: [
  {
    "column_name": "mdr_report_key",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "event_key",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "report_number",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "report_source_code",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "manufacturer_link_flag",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "number_devices_in_event",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "number_patients_in_event",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "date_received"

In [30]:
sql_prompt = (
    f"Based on the following execution steps and table information, generate SQL queries to answer the research question.\n\n"
    #f"MAUDE general Information: {prompt_content}\n\n"
    f"Research Question: {research_question}\n\n"
    f"Execution Steps: {execution_steps}\n\n"
    f"Table Information: {table_info_json}\n\n"
    f"Ensure that table names are formatted as maude.\"TABLENAME\" and use the correct column names as per the table structures.\n\n"
    f"**Constraint**: Please ensure that the SQL query does not return more than 10 hits by limited 10.\n\n"
    f"**Additional Requirement**: Each generated SQL statement should be self-contained and independent, meaning they should not rely on the execution of other SQL statements. If there are dependencies between queries, combine them into a single, cohesive SQL statement."
)
sql_query_full = generate_response(sql_prompt)
if not sql_query_full:
    print("Failed to generate SQL query.")
    cursor.close()
    conn.close()
print(f"Generated SQL Query:\n\n{sql_query_full}\n")


2024-12-08 09:38:47,258 [INFO] HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


Generated SQL Query:

### SQL Queries to Address the Research Question

#### 1. Aggregate Adverse Events by Device Type
```sql
SELECT 
    d.generic_name, 
    d.product_code, 
    COUNT(CASE WHEN m.adverse_event_flag = 'Y' THEN 1 END) AS adverse_event_count,
    COUNT(CASE WHEN m.product_problem_flag = 'Y' THEN 1 END) AS device_problem_count
FROM 
    maude."DEVICE" d
JOIN 
    maude."mdrfoi" m ON d.mdr_report_key = m.mdr_report_key
GROUP BY 
    d.generic_name, d.product_code
LIMIT 10;
```

#### 2. Analyze Device Age Impact
```sql
SELECT 
    d.device_age_text, 
    COUNT(CASE WHEN m.adverse_event_flag = 'Y' THEN 1 END) AS adverse_event_count,
    COUNT(CASE WHEN m.product_problem_flag = 'Y' THEN 1 END) AS device_problem_count
FROM 
    maude."DEVICE" d
JOIN 
    maude."mdrfoi" m ON d.mdr_report_key = m.mdr_report_key
GROUP BY 
    d.device_age_text
LIMIT 10;
```

#### 3. Compare Incidence by Manufacturer
```sql
SELECT 
    d.manufacturer_d_name, 
    COUNT(CASE WHEN m.adverse_event_

In [34]:
import re
pattern = r'```sql\n(.*?)```'
matches = re.findall(pattern, sql_query_full, re.DOTALL)
if matches: # Extract the SQL query and remove any leading/trailing whitespace 
    sql_queries = [match.strip() for match in matches]
    print("Extracted SQL Query:\n")
    # 遍历 sql_queries 数组并逐一打印每个 SQL 查询
    for idx, query in enumerate(sql_queries, start=1):
        print(f"--- SQL Query {idx} ---")
        print(query)
        print("\n")  # 添加空行以增加可读性
else: 
    print("No SQL code block found in sql_query_full.")


Extracted SQL Query:

--- SQL Query 1 ---
SELECT 
    d.generic_name, 
    d.product_code, 
    COUNT(CASE WHEN m.adverse_event_flag = 'Y' THEN 1 END) AS adverse_event_count,
    COUNT(CASE WHEN m.product_problem_flag = 'Y' THEN 1 END) AS device_problem_count
FROM 
    maude."DEVICE" d
JOIN 
    maude."mdrfoi" m ON d.mdr_report_key = m.mdr_report_key
GROUP BY 
    d.generic_name, d.product_code
LIMIT 10;


--- SQL Query 2 ---
SELECT 
    d.device_age_text, 
    COUNT(CASE WHEN m.adverse_event_flag = 'Y' THEN 1 END) AS adverse_event_count,
    COUNT(CASE WHEN m.product_problem_flag = 'Y' THEN 1 END) AS device_problem_count
FROM 
    maude."DEVICE" d
JOIN 
    maude."mdrfoi" m ON d.mdr_report_key = m.mdr_report_key
GROUP BY 
    d.device_age_text
LIMIT 10;


--- SQL Query 3 ---
SELECT 
    d.manufacturer_d_name, 
    COUNT(CASE WHEN m.adverse_event_flag = 'Y' THEN 1 END) AS adverse_event_count,
    COUNT(CASE WHEN m.product_problem_flag = 'Y' THEN 1 END) AS device_problem_count
FROM 
   

In [35]:
import re
import time
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)

# Connect to the database
conn, cursor = connect_database()
if not conn or not cursor:
    logging.error("Database connection failed.")
else:
    logging.info("Successfully connected to the database.")

# Initialize an empty list to collect all data
dataall = []

try:
    # Iterate through each SQL query in the sql_queries list
    for idx, sql_query in enumerate(sql_queries, start=1):
        logging.info(f"Executing SQL Query {idx}/{len(sql_queries)}:\n{sql_query}\n")
        
        attempt = 0
        while attempt < max_retries:
            data, error = execute_sql(cursor, sql_query)
            
            if error:
                logging.error(f"SQL Execution Error on Query {idx}: {error}\n")
                
                # Check if the error is related to transaction state
                if "current transaction is aborted" in error.lower():
                    try:
                        cursor.execute("ROLLBACK;")
                        conn.commit()
                        logging.info("Rolled back the aborted transaction.")
                    except Exception as rollback_error:
                        logging.error(f"Failed to rollback transaction: {rollback_error}")
                        break  # Cannot recover from this state
                    attempt += 1
                    continue  # Retry the same query after rollback
                    
                # Prepare the correction prompt for OpenAI
                correction_prompt = (
                    f"The following SQL query resulted in an error. Please correct it based on the error message.\n\n"
                    f"Error Message: {error}\n\n"
                    f"Original SQL Query:\n{sql_query}\n\n"
                    f"Table Information: {table_info_json}\n\n"
                    f"Provide the corrected SQL query."
                )
                
                # Request OpenAI to correct the SQL query
                corrected_sql = generate_response(correction_prompt)
                
                if not corrected_sql:
                    logging.warning("Failed to correct SQL query. Skipping to the next query.\n")
                    break  # Exit the retry loop and proceed to the next query
                
                logging.info(f"Corrected SQL Query {idx}:\n\n{corrected_sql}\n")
                
                # Extract the SQL query from the corrected response
                pattern = r'```sql\n(.*?)```'
                matches = re.findall(pattern, corrected_sql, re.DOTALL)
                if matches:
                    # Update only the current query with the corrected version
                    corrected_query = matches[0].strip()
                    logging.info(f"Updating Query {idx} with corrected SQL.")
                    sql_query = corrected_query
                else:
                    logging.warning("No SQL code block found in corrected SQL. Skipping to the next query.")
                    break  # Skip to the next query if no correction found
                
                attempt += 1
                time.sleep(1)  # Brief pause before retrying
            else:
                logging.info(f"SQL Query {idx} executed successfully.\n")
                # Append the retrieved data to dataall
                if data:
                    dataall.extend(data)
                    logging.info(f"Retrieved {len(data)} records from Query {idx}.\n")
                else:
                    logging.info(f"No data returned from Query {idx}.\n")
                break  # Exit the retry loop and proceed to the next query
        
        # After max_retries, check if the last attempt resulted in an error
        if error and attempt == max_retries:
            logging.error(f"Reached maximum retry attempts for Query {idx}. Unable to execute this query.\n")
    
finally:
    # Ensure the database connection is closed properly
    try:
        cursor.close()
        conn.close()
        logging.info("Database connection closed.")
    except Exception as close_error:
        logging.error(f"Error closing database connection: {close_error}")

# Optional: Display the aggregated data
logging.info(f"Total records retrieved from all queries: {len(dataall)}")


2024-12-08 09:47:32,000 [INFO] Successfully connected to the database.
2024-12-08 09:47:32,002 [INFO] Executing SQL Query 1/4:
SELECT 
    d.generic_name, 
    d.product_code, 
    COUNT(CASE WHEN m.adverse_event_flag = 'Y' THEN 1 END) AS adverse_event_count,
    COUNT(CASE WHEN m.product_problem_flag = 'Y' THEN 1 END) AS device_problem_count
FROM 
    maude."DEVICE" d
JOIN 
    maude."mdrfoi" m ON d.mdr_report_key = m.mdr_report_key
GROUP BY 
    d.generic_name, d.product_code
LIMIT 10;



Successfully connected to the database.


2024-12-08 09:47:32,275 [ERROR] SQL Execution Error on Query 1: column d.product_code does not exist
LINE 3:     d.product_code, 
            ^


2024-12-08 09:47:32,683 [INFO] HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2024-12-08 09:47:44,231 [INFO] Corrected SQL Query 1:

The error message indicates that the column `d.product_code` does not exist in the `maude."DEVICE"` table. Based on the provided table information, it appears that the correct column name should be `device_report_product_code`.

Here is the corrected SQL query:

```sql
SELECT 
    d.generic_name, 
    d.device_report_product_code, 
    COUNT(CASE WHEN m.adverse_event_flag = 'Y' THEN 1 END) AS adverse_event_count,
    COUNT(CASE WHEN m.product_problem_flag = 'Y' THEN 1 END) AS device_problem_count
FROM 
    maude."DEVICE" d
JOIN 
    maude."mdrfoi" m ON d.mdr_report_key = m.mdr_report_key
GROUP BY 
    d.generic_name, d.device_report_product_code
LIMIT 10;
```

This query uses the 

In [36]:
# 9. Analyze data to validate the research question
if dataall:
    # Construct the formatted string and assign it to a new variable
    tmp = f"{sql_query_full}\n\nSQL Execution Outcome:\n{dataall}"

    analyze_data(research_question, data)

2024-12-08 09:48:07,588 [INFO] HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


Analysis Report:
### Analysis Report: Validity and Feasibility of the Research Question

#### 1. **Research Question Overview**

**Research Question:**
"How do the rates of adverse events and device problems vary across different types of medical devices, and what factors (such as device age, manufacturer, and type of operator) are associated with higher incidence of these events?"

**Rationale:**
The research question aims to explore variations in adverse events and device problems across different medical devices, considering factors like device age, manufacturer, and operator type. The MAUDE (Manufacturer and User Facility Device Experience) database provides comprehensive data for this analysis.

#### 2. **Data Overview**

The provided data includes:
- **Device Operator**: Type of person operating the device.
- **Adverse Event Count**: Number of adverse events reported.
- **Device Problem Count**: Number of device problems reported.

**Data Sample:**
```json
[
    {"device_operator